In [ ]:
import math
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from collections import defaultdict
from scipy.stats.stats import pearsonr
import pandas as pd

In [ ]:
df_tennis = pd.read_csv('dataset/tennis_matches.csv', sep=',', index_col=0) 

#index_col=False indica di non usare la prima colonna come id, ma come dati
df_male = pd.read_csv('dataset/male_players.csv', sep=',', index_col=False)
df_female = pd.read_csv('dataset/female_players.csv', sep=',', index_col=False) 

# Male Table

Surname cleaning, in the surname are present substring with latters and numbers that clearly rappresents errors. The following code clears the surnames form this strings.

In [ ]:
#ci sono cifre nei nomi/cognomi?
import re
re_name = "^^[a-zA-Z]+(([',. -][a-zA-Z ])?[a-zA-Z]*)*$"
wrong_surnames = []
wrong_surnames_indexes = []
for i,row in enumerate(df_male['surname']):
    if not(re.search(re_name, str(row))):
        wrong_surnames.append(str(row))
        wrong_surnames_indexes.append(i)

In [ ]:
wrong_surnames

In [ ]:
#TO DO: Gestire stringa vuota, caso dell'entry con l'1 come cognome (eliminare l'entry?)
correct_surname = []
for wrong_surname in wrong_surnames_indexes:
    words = str(df_male['surname'][wrong_surname]).split(" ")
    #se è =1 togliamo solo il numero dalla parola.
    if len(words)==1:
        for caracter in words[0]:
            if caracter.isdigit():
                words[0] = words[0].replace(str(caracter),"")
        correct_surname.append(words[0])
    #se >1 togliamo la seconda parola se ci sono numeri
    if len(words)>1:
        for word in words:
            if not(re.search(re_name, str(word))):
                words.remove(word)
            #check se tutte le parole hanno len>1: se no toglile
#            if len(word)==1:
 #               words.remove(word)
        correct_surname.append(' '.join(words))

In [ ]:
correct_surname

In [ ]:
#TO DO:Effettuare la correzione dei anche sui nomi della tabella df_tennis